<a href="https://colab.research.google.com/github/andres-merino/Generador-Moodle-LaTeX/blob/main/Codigo%20Colab%20v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generador de preguntas de Moodle

In [11]:
# Paquetes
import itertools
import os
import random
import numpy as np
from scipy.stats import *
from sympy import *

## Definiciones

Llenar estos datos, en el enunciado, los parámetros y datos que deseen sean reemplazados o calculados por el generador, deben estar entre corchetes dobles `[[ ]]`. Dentro de cada pregunta, si un item no tiene opciones, dejar corchetes vacíos `[]` (ver ejemplo). El identificador de la pregunta siempre debe tener el parámetro `[[id]]` para que sea reemplazado por el generador.

In [12]:
# Nombre del cuestionario
name = "Producto"

# Parámetros
par = ["a", "b"]
# Diccionario de parámetros
par_dict = {
    "a": [-2,-1,0,1,2],
    "b": [-2,-1,1,2],
    }

# Enunciado
enunciado = r'''
\begin{numerical}[tolerance=0.01]%
    % - Indentificador
    {Producto - [[id]]}
    % - Enunciado
    Determine el valor de
    \[
        [[a]] + [[b]]
    \]
    \item[] [[a+b]]
\end{numerical}
'''

# Cantidad máxima de ejercicios (opcional)
max_ejercicios = 2
# max_ejercicios = None


## Probar con un parámetro

Para probar con un parámetro, ejecutar el siguiente código. Si se quiere solo generar una cantidad $n$ de preguntas para copiarlas, cambiar el valor de total preguntas.


In [ ]:
# Total preguntas
# _total = 1
_total = 2
# Reemplazo todos los { por {{
_enunciado_F = enunciado.replace('{', '{{')
_enunciado_F = _enunciado_F.replace('}', '}}')
# Reemplazo todos los corchetes por {
_enunciado_F = _enunciado_F.replace('[[', '{')
_enunciado_F = _enunciado_F.replace(']]', '}')

# Lista de parámetros
_par_list = []
for _parametro in par:
    _par_list.append(par_dict[_parametro])

# Semilla
random.seed(10)

# Producto cartesiano
_par_comb = list(itertools.product(*_par_list))
_par_comb = random.sample(_par_comb, _total)

# Genero el examen
_quiz = ''
id = 1
for _parametros in _par_comb:
    # try:
    for _n, _parametro in enumerate(par):
        exec(f"{_parametro} = _parametros[{_n}]")
    # Reemplazo
    _quiz += '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%'
    exec(f'_quiz += fr"""{_enunciado_F}"""')
    _quiz += '\n'
    id += 1
    # except:
        # print('Error en los parámetros:',par, '=', _parametros)
    if id > _total:
            break

# Mensaje
print(_quiz)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{numerical}[tolerance=0.01]%
    % - Indentificador
    {Suma - 1}
    % - Enunciado
    Determine el valor de 
    \[
        2 + 1
    \]
    \item[] 3
\end{numerical}

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{numerical}[tolerance=0.01]%
    % - Indentificador
    {Suma - 2}
    % - Enunciado
    Determine el valor de 
    \[
        -2 + -1
    \]
    \item[] -3
\end{numerical}




## Generar el examen

Si todo está bien, ejecutar el siguiente código para generar el examen.

In [ ]:
# Reemplazo todos los { por {{
_enunciado_F = enunciado.replace('{', '{{')
_enunciado_F = _enunciado_F.replace('}', '}}')
# Reemplazo todos los corchetes por {
_enunciado_F = _enunciado_F.replace('[[', '{')
_enunciado_F = _enunciado_F.replace(']]', '}')

# Lista de parámetros
_par_list = []
for _parametro in par:
    _par_list.append(par_dict[_parametro])

# Producto cartesiano
_par_comb = list(itertools.product(*_par_list))

# Semilla
random.seed(10)

# Si está definido el máximo de ejercicios
if max_ejercicios:
    # Tomo combinaciones aleatorias
    _par_comb = random.sample(_par_comb, max_ejercicios)

# Genero el examen
_quiz = ''
id = 1
for _parametros in _par_comb:
    try:
        for _n, _parametro in enumerate(par):
            exec(f"{_parametro} = _parametros[{_n}]")
        # Reemplazo
        _quiz += '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%'
        exec(f'_quiz += fr"""{_enunciado_F}"""')
        _quiz += '\n'
        id += 1
    except:
        print('Error en los parámetros:',par, '=', _parametros)

_parametros_latex = ''
for _parametro in par:
    _parametros_latex += '\t' + fr'\item ${_parametro} \in ' + '\{' + ', '.join(map(str, par_dict[_parametro])) + '\}$' + '\n'

# Leo el archivo plantilla
_plantilla = open('Plantilla.tex', 'r', encoding='utf-8')
_t_plantilla = _plantilla.read()
_plantilla.close()
# Reemplazo
_t_plantilla = _t_plantilla.replace('{{QUIZ}}', _quiz)
_t_plantilla = _t_plantilla.replace('{{Cuestionario}}', name)
_t_plantilla = _t_plantilla.replace('{{Enunciado}}', enunciado)
_t_plantilla = _t_plantilla.replace('{{Parámetros}}', _parametros_latex)
_t_plantilla = _t_plantilla.replace('{{Número de preguntas}}', str(id-1))

# Escribo el archivo
_examen = open(f'test/{name}.tex', 'w', encoding='utf-8')
_examen.write(_t_plantilla)
_examen.close()

# Compilo el examen con xeLaTeX
os.system(f'latexmk -xelatex "{name}.tex" -output-directory="./test"')

# Elimino todos los archivos llamados examen, salvo el pdf y el xml
_archivos = os.listdir('./test')
for _archivo in _archivos:
    if name in _archivo:
        if '.pdf' not in _archivo and '.xml' not in _archivo and '.tex' not in _archivo:
            try:
                os.remove("test/"+_archivo)
            except:
                pass
# Elimino los archivos fls
for _archivo in _archivos:
    if '.fls' in _archivo:
        try:
            os.remove(_archivo)
        except:
            pass

# Mensaje
print(f'Examen generado con éxito, se generaron {id-1} preguntas.')